# <center><span style="color:red"><u> Grupo 2</u> </span> <center>
    
   <center><i>- Beatriz, Diego, Edwin, Luis -</i></center>

## <center><u>Título: Open-Channel Flow Algorithm in Newton-Raphson Form </u>
<center>by: John N. Paine


### <u>Objetivo</u>: Desenvolver um <u>algorítmo</u> para solucionar um problema de determinação do perfil d'água em um fluxo em regime permanente gradualmente variado baseado no<u> _Standard Step Method_</u>. 


### <u> Justificativa</u>

- O escoamento **permanente gradualmente variado** vem sendo objeto de estudo por mais de <u>100 anos</u>.
- Conhecer o **perfil d'água** é extramemente importante pois <u>influencia diretamente as obras de engenharia</u>.
- Todavia, o problema é extramamente <u>complexo</u>, principalmente pelo ***tratamento matemático das equações*** que regem o movimento e também o **grande número de parâmentros hidráulicos** sujeitos a variações.
- Foram desenvolvidos diversos métodos, com destaque para os trabalho iniciais de **Bakhmeteff**.
- Há diversos métodos para determinação do perfil:
    1. métodos de integração gráfica;
    2. métodos de integração direta;
    3. métodos de soluções numéricas passo a passo (*step methods*):
        1. Direct step method;
        2. Standard step method;

> ### Direct step method
>
> - A distância é calculada a partir da altura;
> - Aplicável somente em canais prismáticos;

> ### Standard step method
> - A altura é calculada a partir da distância;
> - Aplicável em canais prismáticos e não prismáticos;
> - Processo de tentativas e erros
> - Envolve uma equação diferencial, é necessária uma condição de contorno.

<img src="canal.png" style="height:700px">

### <u>Equações</u>

> #### Energia disponível:
>
> $$H = i + y + \alpha \frac{V^2}{2g}$$

   
<br>

> #### Equação básica do standard step, com condições de contorno conhecidas em uma das seções:
>
> $$f(x) = \left( h + \frac{V^2}{2g}\right)_D- \left(h + \frac{V^2}{2g}\right)_U + \frac{\Delta x}{2} (S_{fU} + S_{fD})$$

<br>
   
> #### Gradiente de Energia
>
> $$S_f = \left( \frac{Qn}{AR^{2/3}} \right)^2$$

### <u>Constantes</u>

Algumas equações e passagens são chamadas de constantes **$(C_x)$** para simplificar as equações, sendo essas:

> $$C_1 = H$$
<br>
> $$C_2 = S_f$$
<br>
> $$C_3 = Qn$$
<br>
> $$C_4 = \frac {Z_L + Z_R}{2}$$
<br>
> $$C_5 = \left(1 + Z_L^2\right)^{1/2} + \left(1 + Z_R^2\right)^{1/2}$$

### <u>Equações Simplificadas para Newton-Raphson</u>

#### Função polinomial do gradiente de energia  em função de *y* como variável independente

> $$S_f(y)= C_3^2 \left(B + C_4y^2\right)^{-10/3}\left(B + yC_5\right)^{4/3}$$

<br>

#### Equação final do standard step method com as constantes definidas utilizada no algorítimo

> $$f(y) = \beta C_1 - \beta y - \beta i - \beta \frac {\left[ \frac{Q}{y \left(B + C_4y\right)}\right]^2}{2g} + \frac {\Delta x}{2} \left[S_f(y) + C_2 \right]$$

O termo **$\beta$** é <font color='blue'>positivo</font> se o calculo inicia com valores conhecidos na montante do canal ou <font color='red'>negativo</font> se são conhecidos os valores na jusante do canal.



**Observação:** Quando a função apresenta duas raízes, o valor inicial de $yk$ tem que ser bem proximo do $y$ inicial

### <u>Aplicação do SSM em python</u>

Os indices K são para valores conhecidos e o indice U para os desconhecidos.

**Observação:** Há condições físicas que impedem a solução do Standard Step Method, o comprimento do perfil de fluxo pode ser menor que o comprimento do canal ou a presença de um ressalto hidráulico. Nesses casos tem que se realizar verificações para uma finalização controlada do método. (Script para calcular altura crítica).

### <u>Exemplo de execução</u>


 ### <u>Código do Algoritmo</u>
>
>### Passo 1:
> Importaram-se os módulos necessários para desenvolvimento do código


In [ ]:
from scipy import optimize
import matplotlib.pyplot as plt
import hydradepth as dp


> ### Passo 2:
>
> Definiram-se as equações e constantes auxiliares



In [ ]:
#Equação básica do standard step

def fy(y):
    Sfy = Sf(y)
    C1 = H(iK, yK)
    C2 = SfK(Q, n)
    C4 = inclina(ZL, ZR)
    return BETA*C1 - BETA*y - BETA*i - BETA/(2.0*g)*(Q/(y*(b+C4*y)))**2.0 + (deltax/2.0)*(Sfy + C2)

# Derivada de f(y) (equação básica do método)

def derivada_fy(y):
    derivda_Sfy = SfP(y)
    C4 = inclina(ZL, ZR)
    return BETA*(-1.0) + BETA*(Q*Q/g) * (y*(b + C4*y))**(-3.0) * (b + 2*C4*y) + (deltax/2.0)*(derivda_Sfy)

#Função polinomial do gradiente de energia

def Sf(y):
    C3 = mann(Q, n)
    C4 = inclina(ZL, ZR)
    C5 = raiz_inclina(ZL, ZR)

    return (C3**2) * ((b*y + (C4*y**2.0))**(-10.0/3.0)) * (b + y*C5)**(4.0/3.0)

# Derivada da função polinomial do gradiente de energia

def SfP(y):
    C3 = mann(Q, n)
    C4 = inclina(ZL, ZR)
    C5 = raiz_inclina(ZL, ZR)
    return (4.0/3.0)*C3*C3*C5 * ((b*y + C4*y*y)**(-10.0/3.0)) * (b + C5*y)**(1.0/3.0) - (10.0/3.0)*(C3*C3) * (b + 2*C4*y) * (b + C5*y)**(4.0/3.0) * (y*(b + C4*y))**(-13.0/3.0)

#Energia disponível (constante auxiliar)
def H(iK, yK):  # C1 
    C4 = inclina(ZL, ZR)
    VK = Q/(yK*(b + C4*yK))

    return iK + yK + (VK**2.0)/(2*g)

#Constante auxiliar 
def SfK(Q, n):  # C2
    A = yK*(b + 0.5*yK*(ZL+ZR))
    P = b + (yK**2.0 + yK*ZL*yK*ZL)**0.5 + (yK**2.0 + yK*ZR*yK*ZR)**0.5
    R = A/P
    return (Q*n / (1.486*A*R**(2.0/3.0)))**2.0

# 
def mann(Q, n):  # C3
    return Q*n/1.486  # ft - 1.496; m - 1.000

#Inclinação ( constante auxiliar)

def inclina(ZL, ZR):  # C4
    return (ZL + ZR) / 2.0

# Simplificação para cálculo do perímetro molhado (constante auxiliar)

def raiz_inclina(ZL, ZR):  # C5
    return (1+(ZL)**2.0)**0.5 + (1+(ZR)**2.0)**0.5


> ### Passo 3:
>
> Definiram-se:
> - As características do canal;
> - Os dados do projeto;
> - As condições iniciais de contorno ;
> - A verificação hidráulica para os dados inseridos.

In [ ]:

# channel features

ZL = 2.0
ZR = 1.0
bU = 5.0
bD = 5.0
b = abs((bU + bD)/2)
n = 0.015  # manning's coefficient
L = 25.0
Initial_Station = 50
iDownstream = 35.21
iUpstream = 37.22
slope = (iUpstream - iDownstream) / L

# project features

Q = 285
g = 32.2  # ft/s^2
ns = 20  # number of sections - need to be integer
deltax = L/(ns-1) # distância entre as seções

# boundary condition

iKinicial = 35.21  # inverts know
BETA = 1  # down to up = 1.0; up to down = - 1.0
yKi = 2.15  # y know


# hydraulic verification

args = [b, ZL, ZR, g, Q]
yc = dp.critical(yKi, args)

args0 = [n, Q, slope, b, ZL, ZR]
yn = dp.normal(yKi, args0)


> ### Passo 4: Cálculo dos dados das seções adjacentes

> <b> 1 - </b>  Para resolução do problema dividiu-se o canal em 20 seções. Sendo conhecidos apenas os dados da primeira seção, como apresentado no passo 3.
<br>

> <b> 2 - </b>  A partir dos dados conhecidos da seção inicial (<b> i </b> e <b> y </b>), é feito um chute inicial para cálculo dos dados da seção adjacente. 
<br>
> <b> Observação: </b> A única função desse chute inicial é dar um ponto de início para as interações do método Newton-Raphson
<br>

> <b> 3 - </b>  Os dados da seção calculada (adjacente), serão utilizados como dados iniciais para a seção adjacente seguinte, e esse processo é repetido até que todas as seções sejam calculadas. 


In [ ]:

# create the inverts unknown

iU = ns * [iKinicial]

for x in range(1, ns):  # start in second '[1]' to keep the value of know invert in first section
    iU[x] = deltax * slope + iU[x-1]

# create the stations

station = (ns) * [Initial_Station]
for x in range(0, ns):
    station[x] = Initial_Station + deltax*x

# create the y unknow

y = (ns) * [yKi]

for x in range(1, ns):
    yK = y[x-1]
    iK = iU[x-1]
    i = iU[x]

    try:
        y[x] = optimize.newton(fy, y[x-1]+0.01, fprime=derivada_fy)
    except RuntimeError or ZeroDivisionError:
        y[x] = yc
        del y[x+1:]
        del station[x+1:]
        del iU[x+1:]
        break

# stage depth (invert + depth)

stagedepth = (len(y)) * [Initial_Station]
for x in range(len(y)):
    stagedepth[x] = iU[x] + y[x]

> ### Passo 5:
> Apresentação dos dados


In [ ]:

# critical depth for print

ycrit = (len(y)) * [yc]
for x in range(len(y)):
    ycrit[x] = iU[x] + yc

# normal depth for print

ynorm = (len(y)) * [yn]
for x in range(len(y)):
    ynorm[x] = iU[x] + yn

print("="*36)
print("--- STANDARD STEP METHOD RESULTS ---")
print("="*36)
print('NORMAL DEPTH = ' f'{yn:2.2f}' ' ft')
print('CRITICAL DEPTH = ' f'{yc:2.2f}'' ft')
print("-"*36)
print("STATION       STAGE       FLOW DEPTH")
print("              (ft)           (ft)")
print("-"*36)

for x in range(len(y)):
    print(f' {station[x]:2.2f}        {stagedepth[x]:2.2f}          {y[x]:2.2f}')
print("-"*36)


> ### Passo 6:
> Apresentação gráfica


In [ ]:
# printing graphic
with plt.style.context('Solarize_Light2'):
    plt.rcParams['figure.figsize'] = (15,9)
    plt.plot(station, iU, 'k', label='Invert')
    plt.plot(station, stagedepth, 'b--', label='Flow Depth')
    plt.plot(station, ycrit, 'r:', label='Critical Depth')
    plt.plot(station, ynorm, 'g-.', label='Normal Depth')

    legend = plt.legend(loc='best', shadow=True, fontsize='small')

    legend.get_frame().set_facecolor('#AAAC7B')
    plt.xlabel('STATION')
    plt.ylabel('DEPTH')
    plt.title('STANDARD STEP')
    plt.show()

 ### <u>Conclusões  </u>
  
  1. Um algorítimo foi escrito e desenvolvido em python apresentando resultados em velocidade muito mais rápida do que em relação a rotinas de interpolação com consulta a tabelas.
  2. Apesar do autor original indicar que o algorítmo poderia funcionar em qualquer tipo de regime, na prática isso não ocorre. Em momentos em que o $\Delta x$ se tornam muito pequenos em relação a altura $y$, ou em transição de regime o algorítmo, por uma limitação física / matemática não consegue apresentar resultados. Desse modo é necessário inserir uma rotina de verificação de erro.
  3. O método permite o calculo em canais prismáticos e não prismáticos, todavia este exemplo foi estruturado para canais trapezoidais. Canais com a geometria mais complexa, como canais com leito natural, exigirão um módulo a parte para o cálculo da Am e Rh.
  4. Por conta desta última limitação e por considerar um coeficiente de Manning constante em todo o canal, a rotina desenvolvida atende muito bem os canais abertos encontrados na maior dos sistemas de drenagem.